## Trying with sample recommendation system

In [ ]:
!pip install transformers


In [ ]:
from transformers import pipeline


In [ ]:
# Load pre-trained emotion detection model
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## First trying with sample songs list

In [ ]:
def detect_emotion(user_input):
    result = emotion_classifier(user_input)
    # Extracting the emotion with the highest score
    emotion = max(result, key=lambda x: x['score'])['label']

    # Mapping detected emotions to our song database keys
    emotion_mapping = {
        'joy': 'happy',
        'sadness': 'sad',
        'anger': 'angry',
        'surprise': 'happy',  # Could be mapped to different emotions
        'love': 'happy',
        'fear': 'sad',
        'disgust': 'angry',
        'neutral': 'relaxed'
    }

    return emotion_mapping.get(emotion, 'neutral')


In [ ]:
song_database = {
    'happy': ["Happy by Pharrell Williams", "Uptown Funk by Bruno Mars"],
    'sad': ["Someone Like You by Adele", "Stay by Rihanna"],
    'angry': ["Break Stuff by Limp Bizkit", "Killing in the Name by Rage Against the Machine"],
    'relaxed': ["Weightless by Marconi Union", "Strawberry Swing by Coldplay"],
    # Add more emotions and songs
}

def recommend_songs(emotion):
    return song_database.get(emotion, ["No recommendations available for this emotion"])


In [ ]:
def chatbot():
    print("Hello! I'm here to recommend you songs based on how you feel.")
    while True:
        user_input = input("Tell me how you're feeling: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break
        emotion = detect_emotion(user_input)
        print(f"I detected that you're feeling {emotion}.")
        songs = recommend_songs(emotion)
        print("Here are some song recommendations for you:")
        for song in songs:
            print(f"- {song}")

# Run the chatbot
chatbot()


Hello! I'm here to recommend you songs based on how you feel.
Tell me how you're feeling: happy
I detected that you're feeling happy.
Here are some song recommendations for you:
- Happy by Pharrell Williams
- Uptown Funk by Bruno Mars
Tell me how you're feeling: exit
Goodbye!


# Using Youtibe for recommendation


In [ ]:
import googleapiclient.discovery
from transformers import pipeline

# YouTube API credentials
api_key = 'your-youtube-api'

# Load pre-trained emotion detection model
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

def detect_emotion(user_input):
    results = emotion_classifier(user_input)
    primary_emotion = results[0]['label']
    return primary_emotion

def map_emotion_to_query(emotion):
    if emotion in ['admiration', 'amusement', 'approval', 'joy', 'excitement', 'gratitude', 'love', 'relief', 'pride']:
        return 'happy songs'
    elif emotion in ['anger', 'annoyance', 'disapproval', 'disgust']:
        return 'angry songs'
    elif emotion in ['sadness', 'disappointment', 'grief', 'remorse', 'embarrassment', 'nervousness', 'fear', 'confusion']:
        return 'sad songs'
    else:
        return 'relaxing music'

def get_youtube_recommendations(query):
    # Build the YouTube service
    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

    # Search for videos
    request = youtube.search().list(
        part="snippet",
        maxResults=20,
        q=query,
        type='video'
    )
    response = request.execute()

    videos = [f"{item['snippet']['title']} (https://www.youtube.com/watch?v={item['id']['videoId']})" for item in response['items']]
    return videos if videos else ["No recommendations available for this emotion"]

def chatbot():
    print("Hello! I'm here to recommend you songs based on how you feel.")
    while True:
        user_input = input("Tell me how you're feeling (describe in sentences or passage): ")
        emotion = detect_emotion(user_input)
        print(f"I detected that you're feeling {emotion}.")

        query = map_emotion_to_query(emotion)
        songs = get_youtube_recommendations(query)

        print(f"Here are some popular song recommendations for you based on your mood:")
        for index, song in enumerate(songs, start=1):
            print(f"{index}. {song}")

        exit_choice = input("Do you want to exit? (yes/no): ").lower()
        if exit_choice == 'yes':
            print("Goodbye!")
            break

if __name__ == "__main__":
    chatbot()


Hello! I'm here to recommend you songs based on how you feel.
Tell me how you're feeling (describe in sentences or passage): My day has been quite productive, filled with engaging conversations and interesting topics to explore.
I detected that you're feeling excitement.
Here are some popular song recommendations for you based on your mood:
1. Relaxing Jazz Cafe Music &amp; Soft Jazz Instrumental Music with June Bossa Nova Piano for Upbeat Moods (https://www.youtube.com/watch?v=udhDi1lwcsM)
2. Pharrell Williams - Happy (Video) (https://www.youtube.com/watch?v=ZbZSe6N_BXs)
3. Imogen Heap - The Happy Song - Official Music Video (https://www.youtube.com/watch?v=9PhtxqKLvVc)
4. My Happy Song | + More Kids Songs | Super Simple Songs (https://www.youtube.com/watch?v=RE29EUeJAb0)
5. Tuesday Morning Jazz - Instrumental Smooth Jazz Music &amp; Relaxing Rhythmic Bossa Nova for a Good Mood (https://www.youtube.com/watch?v=MoWNEHOxFaA)
6. Bobby McFerrin - Don&#39;t Worry Be Happy (Official Music V

In [ ]:
import googleapiclient.discovery
from transformers import pipeline

# YouTube API credentials
api_key = 'youtube-api'

# Load pre-trained emotion detection model
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

def detect_emotion(user_input):
    # Detect emotion using the classifier
    results = emotion_classifier(user_input)
    # Get the most likely emotion
    primary_emotion = results[0]['label']
    return primary_emotion

def map_emotion_to_query(emotion):
    if emotion in ['admiration', 'amusement', 'approval', 'joy', 'excitement', 'gratitude', 'love', 'relief', 'pride']:
        return 'happy songs'
    elif emotion in ['anger', 'annoyance', 'disapproval', 'disgust']:
        return 'angry songs'
    elif emotion in ['sadness', 'disappointment', 'grief', 'remorse', 'embarrassment', 'nervousness', 'fear', 'confusion']:
        return 'sad songs'
    else:
        return 'relaxing music'

def get_youtube_recommendations(query, language=None):
    # Build the YouTube service
    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

    # Modify the query to include language if specified
    if language and language.lower() != 'i am feeling lucky':
        query = f"{query} {language} songs"

    # Search for videos
    request = youtube.search().list(
        part="snippet",
        maxResults=5,
        q=query,
        type='video'
    )
    response = request.execute()

    # Extract video details
    videos = [f"{item['snippet']['title']} (https://www.youtube.com/watch?v={item['id']['videoId']})" for item in response['items']]

    return videos if videos else ["No recommendations available for this emotion and language"]

def chatbot():
    print("Hello! I'm here to recommend you songs based on how you feel.")
    while True:
        user_input = input("Tell me how you're feeling (describe in sentences or passage): ")
        emotion = detect_emotion(user_input)
        print(f"I detected that you're feeling {emotion}.")

        language = input("Which language music do you prefer? (type 'I am feeling lucky' for any language): ").strip()

        query = map_emotion_to_query(emotion)
        songs = get_youtube_recommendations(query, language)

        print(f"Here are some popular song recommendations for you based on your mood and language preference:")
        for index, song in enumerate(songs, start=1):
            print(f"{index}. {song}")

        exit_choice = input("Do you want to exit? (yes/no): ").lower()
        if exit_choice == 'yes':
            print("Goodbye!")
            break

# Run the chatbot
if __name__ == "__main__":
    chatbot()

Hello! I'm here to recommend you songs based on how you feel.
Tell me how you're feeling (describe in sentences or passage): My day has been quite productive, filled with engaging conversations and interesting topics to explore.
I detected that you're feeling excitement.
Which language music do you prefer? (type 'I am feeling lucky' for any language): Korean
Here are some popular song recommendations for you based on your mood and language preference:
1. | Kpop Playlist | Energetic Iconic Songs To Dance To (https://www.youtube.com/watch?v=QcLy_cMr5tg)
2. Top 10 best K-pop songs of 2023 #shorts (https://www.youtube.com/watch?v=QvAsqbn-Fg4)
3. kpop popular/iconic songs to dance to | a playlist (https://www.youtube.com/watch?v=ZiefwX4c-Jo)
4. kpop playlist songs that will make you feel savage (https://www.youtube.com/watch?v=4skab1_3Nrs)
5. เพลงเกาหลีน่ารัก ๆ ฟังสบาย ๆ | Chill Korean Songs/듣기 편한 한국 노래 (https://www.youtube.com/watch?v=Ctg1YOjZios)
Do you want to exit? (yes/no): no
Tell me 

# Youtube recommendation based on Likes

In [ ]:
import googleapiclient.discovery
from transformers import pipeline

# YouTube API credentials
api_key = 'your-api'

# Load pre-trained emotion detection model
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

def detect_emotion(user_input):
    # Detect emotion using the classifier
    results = emotion_classifier(user_input)
    # Get the most likely emotion
    primary_emotion = results[0]['label']
    return primary_emotion

def map_emotion_to_query(emotion):
    if emotion in ['admiration', 'amusement', 'approval', 'joy', 'excitement', 'gratitude', 'love', 'relief', 'pride']:
        return 'happy songs'
    elif emotion in ['anger', 'annoyance', 'disapproval', 'disgust']:
        return 'angry songs'
    elif emotion in ['sadness', 'disappointment', 'grief', 'remorse', 'embarrassment', 'nervousness', 'fear', 'confusion']:
        return 'sad songs'
    else:
        return 'relaxing music'

def get_youtube_recommendations(query, language=None):
    # Build the YouTube service
    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

    # Modify the query to include language if specified
    if language and language.lower() != 'i am feeling lucky':
        query = f"{query} {language} songs"

    # Search for videos
    search_request = youtube.search().list(
        part="snippet",
        maxResults=20,
        q=query,
        type='video'
    )
    search_response = search_request.execute()

    # Extract video IDs
    video_ids = [item['id']['videoId'] for item in search_response['items']]

    # Get video details (including likes)
    video_request = youtube.videos().list(
        part="snippet,statistics",
        id=','.join(video_ids)
    )
    video_response = video_request.execute()

    # Extract video details and sort by likes
    videos = sorted(video_response['items'], key=lambda x: int(x['statistics'].get('likeCount', 0)), reverse=True)

    # Extract top 5 videos
    top_videos = [f"{item['snippet']['title']} (https://www.youtube.com/watch?v={item['id']}) with {item['statistics'].get('likeCount', 0)} likes" for item in videos[:5]]

    return top_videos if top_videos else ["No recommendations available for this emotion and language"]

def chatbot():
    print("Hello! I'm here to recommend you songs based on how you feel.")
    while True:
        user_input = input("Tell me how you're feeling (describe in sentences or passage): ")
        emotion = detect_emotion(user_input)
        print(f"I detected that you're feeling {emotion}.")

        language = input("Which language music do you prefer? (type 'I am feeling lucky' for any language): ").strip()

        query = map_emotion_to_query(emotion)
        songs = get_youtube_recommendations(query, language)

        print(f"Here are the top 5 popular song recommendations for you based on your mood and language preference:")
        for index, song in enumerate(songs, start=1):
            print(f"{index}. {song}")

        exit_choice = input("Do you want to exit? (yes/no): ").lower()
        if exit_choice == 'yes':
            print("Goodbye!")
            break

# Run the chatbot
if __name__ == "__main__":
    chatbot()


Hello! I'm here to recommend you songs based on how you feel.
I detected that you're feeling sadness.
Here are the top 5 popular song recommendations for you based on your mood and language preference:
1. Tum Hi Aana Full Video | Marjaavaan | Riteish D, Sidharth M, Tara S | Jubin N | Payal Dev Kunaal V (https://www.youtube.com/watch?v=RemShT6JAHw) with 3239367 likes
2. "Kabira Full Song" Yeh Jawaani Hai Deewani | Pritam | Ranbir Kapoor, Deepika Padukone (https://www.youtube.com/watch?v=jHNNMj5bNQw) with 2569548 likes
3. Ajay-Atul - Abhi Mujh Mein Kahin Best Lyric|Agneepath|Priyanka Chopra,Hrithik|Sonu Nigam (https://www.youtube.com/watch?v=oWKgpB2zpgw) with 1818776 likes
4. "Kyun Main Jaagoon" Full Song Patiala House | Akshay Kumar (https://www.youtube.com/watch?v=lN1m7zLBbSU) with 1000088 likes
5. Night 🌃 sad 💔songs for sleeping broken heart❤️‍🩹 | slowed + reverb mix | lofi hindi bollywood song (https://www.youtube.com/watch?v=LGmqw3yT0NE) with 980659 likes
I detected that you're feel

## Using a dataset

In [ ]:
import pandas as pd
from textblob import TextBlob
from transformers import pipeline

# Load the dataset
file_path = '/content/data_moods.csv'
df = pd.read_csv(file_path)

# Load pre-trained emotion detection model
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

def detect_emotion(user_input):
    # Use TextBlob for sentence tokenization
    blob = TextBlob(user_input)
    sentences = blob.sentences

    # Aggregate sentiments across sentences
    sentiment_scores = [sentence.sentiment.polarity for sentence in sentences]
    overall_sentiment = sum(sentiment_scores) / len(sentiment_scores)

    # Determine emotion based on overall sentiment
    if overall_sentiment > 0.2:
        return 'Happy'
    elif overall_sentiment < -0.2:
        return 'Sad'
    elif overall_sentiment < 0:
        return 'Angry'
    else:
        return 'Calm'

def get_song_recommendations(emotion):
    # Filter dataset based on emotion
    recommendations = df[df['mood'].str.contains(emotion, case=False, na=False)]

    # Sort by popularity and get the top 10 songs
    top_songs = recommendations.sort_values(by='popularity', ascending=False).head(10)

    # Extract song details
    songs = top_songs[['name', 'artist']].values.tolist()
    song_list = [f"{name} by {artist}" for name, artist in songs]

    return song_list if song_list else ["No recommendations available for this emotion"]

def chatbot():
    print("Hello! I'm here to recommend you songs based on how you feel.")
    while True:
        user_input = input("Tell me how you're feeling (describe in sentences or passage): ")
        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break

        emotion = detect_emotion(user_input)
        print(f"I detected that you're feeling {emotion}.")

        songs = get_song_recommendations(emotion)

        print("Here are some song recommendations for you:")
        for index, song in enumerate(songs, start=1):
            print(f"{index}. {song}")

# Run the chatbot
chatbot()


Hello! I'm here to recommend you songs based on how you feel.
Tell me how you're feeling (describe in sentences or passage): I passed my exams today
I detected that you're feeling Calm.
Here are some song recommendations for you:
1. Lost by Annelie
2. Curiosity by Beau Projet
3. Escaping Time by Benjamin Martins
4. Just Look at You by 369
5. Vague by Amaranth Cove
6. What You Love You Must Love Now by The Six Parts Seven
7. alpha waves by Eucalyptic
8. Mil Sonidos by Reina del Mar
9. The Choice by Gustavo Santaolalla
10. Coda by Jakob Ahlbom
Tell me how you're feeling (describe in sentences or passage): I broke up with my boyfriend
I detected that you're feeling Calm.
Here are some song recommendations for you:
1. Lost by Annelie
2. Curiosity by Beau Projet
3. Escaping Time by Benjamin Martins
4. Just Look at You by 369
5. Vague by Amaranth Cove
6. What You Love You Must Love Now by The Six Parts Seven
7. alpha waves by Eucalyptic
8. Mil Sonidos by Reina del Mar
9. The Choice by Gustav

In [ ]:
df = pd.read_csv(file_path)

# Load pre-trained emotion detection model
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

def detect_emotion(user_input):
    # Detect emotion using the classifier
    results = emotion_classifier(user_input)
    # Get the most likely emotion
    primary_emotion = results[0]['label']
    return primary_emotion

def get_song_recommendations(emotion):
    # Filter dataset based on emotion
    recommendations = df[df['mood'].str.contains(emotion, case=False, na=False)]

    # Sort by popularity and get the top 10 songs
    top_songs = recommendations.sort_values(by='popularity', ascending=False).head(10)

    # Extract song details
    songs = top_songs[['name', 'artist']].values.tolist()
    song_list = [f"{name} by {artist}" for name, artist in songs]

    return song_list if song_list else ["No recommendations available for this emotion"]

def chatbot():
    print("Hello! I'm here to recommend you songs based on how you feel.")
    while True:
        user_input = input("Tell me how you're feeling (describe in sentences or passage): ")
        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break

        emotion = detect_emotion(user_input)
        print(f"I detected that you're feeling {emotion}.")

        songs = get_song_recommendations(emotion)

        print("Here are some song recommendations for you:")
        for index, song in enumerate(songs, start=1):
            print(f"{index}. {song}")

# Run the chatbot
chatbot()

Hello! I'm here to recommend you songs based on how you feel.
Tell me how you're feeling (describe in sentences or passage): I passed my exams today 
I detected that you're feeling sadness.
Here are some song recommendations for you:
1. No recommendations available for this emotion
Tell me how you're feeling (describe in sentences or passage): I got good marks 
I detected that you're feeling joy.
Here are some song recommendations for you:
1. No recommendations available for this emotion
Tell me how you're feeling (describe in sentences or passage): exit
Goodbye!


In [ ]:
unique_moods = df['mood'].unique()


In [ ]:
print(unique_moods)

['Happy' 'Sad' 'Energetic' 'Calm']
